<h1 align=center><font size = 6>Exploring and Clustering Neighborhoods in Toronto</font></h1>
<h1 align=center><font size = 4>Part 1 of the Coursera Last Project for Data Science</font></h1>

### 1. Importing libraries 

In [1]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
%config IPCompleter.greedy=True
from geopy.geocoders import Nominatim
import numpy as np
import pandas as pd
import json
import folium
from pandas.io.json import json_normalize
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

### 2. Data scraping from given url

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
d = pd.read_html(url)

### 3. Raw Dataframe creation using scraped data

In [3]:
df = d[0]
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


### 4. Data processing and data wrangling ( Ignoring cells with Borough = Not Assigned )


In [4]:
df = df[df.Borough !='Not assigned']

In [5]:
df.reset_index(drop = True, inplace = True)
df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Not assigned
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


### 5. Using GroupBy and Join method to group and combine data with the same Postcode, on the same row separated by ","


In [6]:
df1 = df.groupby(['Postcode','Borough']).aggregate(','.join)

In [7]:
df1.reset_index(inplace = True)

In [8]:
df1

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [9]:
df2 = df1

### 6. Assigning the same value of Borough to Neighborhood where Neighborhood = Not assigned


In [10]:
df2.loc[df2['Neighbourhood'].eq('Not assigned'),'Neighbourhood'] = df2.loc[df2['Neighbourhood'].eq('Not assigned'),'Borough']

In [11]:
df2

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


### 7. Using .shape method to print the number of rows of the Dataframe

In [12]:
df2.shape

(103, 3)

### 8. Importing Latitude and Longitude for each postal code into a new Dataframe

In [14]:
df3 = pd.read_csv('https://cocl.us/Geospatial_data')

In [15]:
df3.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df4 = df2

In [17]:
df4.rename(columns ={'Postcode':'Postal Code'},inplace = True)

### 9. Merging Dataframes into 1 Dataframe having Latitude and Longitude

In [18]:
dfwll = pd.merge(df3, df4, on=['Postal Code'], how='inner')
dfwll = dfwll[['Postal Code','Borough','Neighbourhood','Latitude','Longitude']]

In [19]:
dfwll.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### 10. Obtaining Latitude and Longitude of Toronto and Mapping the neighbourhoods on the map

In [20]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [21]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfwll['Latitude'], dfwll['Longitude'], dfwll['Borough'], dfwll['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 11. Creating a Dataframe having the string Toronto contained in Borough column

In [28]:
toronto_data = dfwll[dfwll['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


### 12. Using credentials of foursquare to obtain venues based on Latitude and Longitude

In [29]:
CLIENT_ID = 'IRJQNREKPO3OYLT1NXV05WJ33KXJPTNOSJWGEFHOVKSOJ151' 
CLIENT_SECRET = 'DGASMTE1KONY0DZ5Y3LH1XPMKON3TCE1NUFIRO4H4L2WDOFD' 
VERSION = '20180605' # Foursquare API version

In [30]:
toronto_data.loc[10,'Neighbourhood']

'Rosedale'

In [31]:
neighborhood_latitude = toronto_data.loc[10, 'Latitude']
neighborhood_longitude = toronto_data.loc[10, 'Longitude'] 
neighborhood_name = toronto_data.loc[10, 'Neighbourhood'] 
print('Latitude and Longitude values of {} are {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and Longitude values of Rosedale are 43.6795626, -79.37752940000001.


### 13. Setting a limit of 100 venues in a radius of 500 meters from the giveng location

In [32]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=IRJQNREKPO3OYLT1NXV05WJ33KXJPTNOSJWGEFHOVKSOJ151&client_secret=DGASMTE1KONY0DZ5Y3LH1XPMKON3TCE1NUFIRO4H4L2WDOFD&v=20180605&ll=43.6795626,-79.37752940000001&radius=500&limit=100'

In [33]:
results_of_toronto_500 = requests.get(url).json()
results_of_toronto_500

{'meta': {'code': 200, 'requestId': '5e2eb7e147b43d0dca786c73'},
 'response': {'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.6840626045, 'lng': -79.37131878274371},
   'sw': {'lat': 43.675062595499995, 'lng': -79.38374001725632}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4aff2d47f964a520743522e3',
       'name': 'Rosedale Park',
       'location': {'address': '38 Scholfield Ave.',
        'crossStreet': 'at Edgar Ave.',
        'lat': 43.68232820227814,
        'lng': -79.37893434347683,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.68232820227814,
          'lng': -79.37893434347683}],
        'distance': 32

### 14. Setting a limit of 100 venues in a radius of 1000 meters from the giveng location

In [34]:
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=IRJQNREKPO3OYLT1NXV05WJ33KXJPTNOSJWGEFHOVKSOJ151&client_secret=DGASMTE1KONY0DZ5Y3LH1XPMKON3TCE1NUFIRO4H4L2WDOFD&v=20180605&ll=43.6795626,-79.37752940000001&radius=1000&limit=100'

In [35]:
results_of_toronto_1000 = requests.get(url).json()
results_of_toronto_1000

{'meta': {'code': 200, 'requestId': '5e2eb6fbdd0f85001c817849'},
 'response': {'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 25,
  'suggestedBounds': {'ne': {'lat': 43.68856260900001,
    'lng': -79.36510816548741},
   'sw': {'lat': 43.670562590999985, 'lng': -79.38995063451262}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4adcb343f964a520e32e21e3',
       'name': 'Summerhill Market',
       'location': {'address': '446 Summerhill Ave',
        'crossStreet': 'btwn. MacLennan Ave. and Glen Rd.',
        'lat': 43.68626482142425,
        'lng': -79.37545823237794,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.68626482142425,
          'lng': -79.3754582

In [36]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [37]:
venues_500 = results_of_toronto_500['response']['groups'][0]['items']
    
nearby_venues_500 = json_normalize(venues_500) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_500 =nearby_venues_500.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_500['venue.categories'] = nearby_venues_500.apply(get_category_type, axis=1)

# clean columns
nearby_venues_500.columns = [col.split(".")[-1] for col in nearby_venues_500.columns]

nearby_venues_500.head()

,name,categories,lat,lng
0,Rosedale Park,Playground,43.682328,-79.378934
1,Whitney Park,Park,43.682036,-79.373788
2,Alex Murray Parkette,Park,43.678300,-79.382773
3,Milkman's Lane,Trail,43.676352,-79.373842


In [38]:
venues_1000 = results_of_toronto_1000['response']['groups'][0]['items']
    
nearby_venues_1000 = json_normalize(venues_1000) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_1000 =nearby_venues_1000.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_1000['venue.categories'] = nearby_venues_1000.apply(get_category_type, axis=1)

# clean columns
nearby_venues_1000.columns = [col.split(".")[-1] for col in nearby_venues_1000.columns]

nearby_venues_1000.head()

,name,categories,lat,lng
0,Summerhill Market,Grocery Store,43.686265,-79.375458
1,Toronto Lawn Tennis Club,Athletics & Sports,43.680667,-79.388559
2,Black Camel,BBQ Joint,43.677016,-79.389367
3,Pie Squared,Pie Shop,43.672143,-79.377856
4,Tinuno,Filipino Restaurant,43.671281,-79.374920


In [40]:
print('{} venues were returned by Foursquare with a radius of 500 m.'.format(nearby_venues_500.shape[0]))
print('{} venues were returned by Foursquare with a radius of 1000 m.'.format(nearby_venues_1000.shape[0]))

4 venues were returned by Foursquare with a radius of 500 m.
25 venues were returned by Foursquare with a radius of 1000 m.


### 15. Analyzing all the venues in a radius of 1000 m related to neighbourhoods

In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_toronto=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results_all_neighbourhoods = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_toronto.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results_all_neighbourhoods])

    nearby_venues_neighbourhood = pd.DataFrame([item for venue_list in venues_toronto for item in venue_list])
    nearby_venues_neighbourhood.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues_neighbourhood)

In [42]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

In [44]:
print(toronto_venues.shape)
toronto_venues.head()

(3185, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
2,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
3,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
4,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop


In [45]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Brockton,Exhibition Place,Parkdale Village",100,100,100,100,100,100
Business Reply Mail Processing Centre 969 Eastern,47,47,47,47,47,47
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",15,15,15,15,15,15
"Cabbagetown,St. James Town",35,35,35,35,35,35
Central Bay Street,100,100,100,100,100,100
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,100,100,100,100,100,100


In [48]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 271 uniques categories.


In [46]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [49]:
toronto_onehot.shape

(3185, 272)

### 17. Grouping the venues based on Neighbourhoods

In [50]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Adelaide,King,Richmond",0.00,0.00,0.000000,0.000000,0.020000,0.00,0.00,0.00,0.00,0.000000,0.010000,0.010000

In [52]:
toronto_grouped.shape

(39, 272)

### 18. Printing the top 5 venues category based on frequency in neighbourhood

In [53]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0         Café  0.06
1  Coffee Shop  0.06
2        Hotel  0.05
3      Theater  0.04
4   Steakhouse  0.03


----Berczy Park----
         venue  freq
0  Coffee Shop  0.11
1         Café  0.06
2        Hotel  0.05
3   Restaurant  0.04
4     Beer Bar  0.04


----Brockton,Exhibition Place,Parkdale Village----
         venue  freq
0         Café  0.07
1  Coffee Shop  0.06
2          Bar  0.05
3   Restaurant  0.04
4       Bakery  0.04


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0         Park  0.09
1  Coffee Shop  0.09
2  Pizza Place  0.06
3      Brewery  0.06
4    Pet Store  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
             venue  freq
0             Café  0.13
1      Coffee Shop  0.13
2  Harbor / Marina  0.13
3            Track  0.07
4   Scenic Lookout  0.07


----Cabbagetown,St. James Town----
                 venue  freq
0  

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### 19. Insert top 10 most common venues for each neighbourhood

In [62]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']
for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Café,Coffee Shop,Hotel,Theater,Restaurant,Cosmetics Shop,Steakhouse,Ramen Restaurant,Pizza Place,Gym
1,Berczy Park,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Beer Bar,Park,Cocktail Bar,Seafood Restaurant,Gastropub
2,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Bar,Restaurant,Bakery,Furniture / Home Store,Tibetan Restaurant,Lounge,Gift Shop,Juice Bar
3,Business Reply Mail Processing Centre 969 Eastern,Park,Coffee Shop,Pizza Place,Brewery,Pet Store,Sushi Restaurant,Italian Restaurant,Burrito Place,Fish & Chips Shop,Bistro
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Café,Harbor / Marina,Coffee Shop,Dance Studio,Sculpture Garden,Dog Run,Scenic Lookout,Park,Track,Garden
5,"Cabbagetown,St. James Town",Gastropub,Café,Diner,Japanese Restaurant,Park,Coffee Shop,Thai Restaurant,Italian Restaurant,Theater,Restaurant
6,Central Bay Street,Coffee Shop,Café,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Ice Cream Shop,Burger Joint,Burrito Place,Bakery,Bookstore
7,"Chinatown,Grange Park,Kensington Market",Café,Vegetarian / Vegan Restaurant,Bar,Art Gallery,Vietnamese Restaurant,Dessert Shop,Coffee Shop,Mexican Restaurant,Tea Room,French Restaurant
8,Christie,Korean Restaurant,Café,Coffee Shop,Grocery Store,Ice Cream Shop,Cocktail Bar,Mexican Restaurant,Pizza Place,Comedy Club,Bar
9,Church and Wellesley,Coffee Shop,Park,Japanese Restaurant,Men's Store,Café,Gym,Italian Restaurant,Restaurant,Hotel,Gay Bar


### 20. Clustering neighbourhoods in 5 clusters

In [63]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 1, 3, 0, 4, 4, 1, 1, 4], dtype=int32)

In [64]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Pizza Place,Coffee Shop,Japanese Restaurant,Breakfast Spot,Beach,Tea Room,Bar,Bakery,Sandwich Place
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Pub,Café,Italian Restaurant,Pizza Place,Spa,Ice Cream Shop,Fast Food Restaurant,Discount Store
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,1,Indian Restaurant,Café,Coffee Shop,Beach,Burrito Place,Bakery,Sandwich Place,Fast Food Restaurant,Butcher,Burger Joint
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Coffee Shop,Bar,Café,American Restaurant,Vietnamese Restaurant,Brewery,Bakery,Italian Restaurant,French Restaurant,Diner
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Café,Coffee Shop,Park,Bookstore,College Quad,Gym / Fitness Center,College Gym,Trail,Dumpling Restaurant,Eastern European Restaurant
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,4,Coffee Shop,Italian Restaurant,Café,Fast Food Restaurant,Pizza Place,Gym,Sushi Restaurant,Dessert Shop,Mexican Restaurant,Bookstore
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,3,Italian Restaurant,Sporting Goods Shop,Skating Rink,Coffee Shop,Park,Fast Food Restaurant,Café,Diner,Mexican Restaurant,Spa
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,4,Coffee Shop,Italian Restaurant,Sushi Restaurant,Indian Restaurant,Middle Eastern Restaurant,Café,Pub,Gym,Pizza Place,Dessert Shop
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,3,Coffee Shop,Grocery Store,Italian Restaurant,Thai Restaurant,Gym,Park,Sushi Restaurant,Pizza Place,Café,Pub
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,3,Coffee Shop,Park,Sushi Restaurant,Italian Restaurant,Grocery Store,Gym / Fitness Center,Thai Restaurant,Pizza Place,Liquor Store,Café


In [65]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 21. Showing each cluster based on its label

In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Downtown Toronto,0,Café,Harbor / Marina,Coffee Shop,Dance Studio,Sculpture Garden,Dog Run,Scenic Lookout,Park,Track,Garden


In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,Pub,Pizza Place,Coffee Shop,Japanese Restaurant,Breakfast Spot,Beach,Tea Room,Bar,Bakery,Sandwich Place
1,East Toronto,1,Greek Restaurant,Coffee Shop,Pub,Café,Italian Restaurant,Pizza Place,Spa,Ice Cream Shop,Fast Food Restaurant,Discount Store
2,East Toronto,1,Indian Restaurant,Café,Coffee Shop,Beach,Burrito Place,Bakery,Sandwich Place,Fast Food Restaurant,Butcher,Burger Joint
3,East Toronto,1,Coffee Shop,Bar,Café,American Restaurant,Vietnamese Restaurant,Brewery,Bakery,Italian Restaurant,French Restaurant,Diner
24,Central Toronto,1,Vegetarian / Vegan Restaurant,Café,Pub,Italian Restaurant,Coffee Shop,Mexican Restaurant,Gym,Grocery Store,Restaurant,Bakery
25,Downtown Toronto,1,Café,Bar,Vegetarian / Vegan Restaurant,Bakery,Bookstore,Restaurant,Mexican Restaurant,Coffee Shop,Burger Joint,Pizza Place
26,Downtown Toronto,1,Café,Vegetarian / Vegan Restaurant,Bar,Art Gallery,Vietnamese Restaurant,Dessert Shop,Coffee Shop,Mexican Restaurant,Tea Room,French Restaurant
30,Downtown Toronto,1,Korean Restaurant,Café,Coffee Shop,Grocery Store,Ice Cream Shop,Cocktail Bar,Mexican Restaurant,Pizza Place,Comedy Club,Bar
32,West Toronto,1,Café,Bar,Restaurant,Bakery,Coffee Shop,Pizza Place,Cocktail Bar,Italian Restaurant,Men's Store,Vegetarian / Vegan Restaurant
33,West Toronto,1,Café,Coffee Shop,Bar,Restaurant,Bakery,Furniture / Home Store,Tibetan Restaurant,Lounge,Gift Shop,Juice Bar


In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Café,Coffee Shop,Park,Bookstore,College Quad,Gym / Fitness Center,College Gym,Trail,Dumpling Restaurant,Eastern European Restaurant


In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Toronto,3,Italian Restaurant,Sporting Goods Shop,Skating Rink,Coffee Shop,Park,Fast Food Restaurant,Café,Diner,Mexican Restaurant,Spa
8,Central Toronto,3,Coffee Shop,Grocery Store,Italian Restaurant,Thai Restaurant,Gym,Park,Sushi Restaurant,Pizza Place,Café,Pub
9,Central Toronto,3,Coffee Shop,Park,Sushi Restaurant,Italian Restaurant,Grocery Store,Gym / Fitness Center,Thai Restaurant,Pizza Place,Liquor Store,Café
10,Downtown Toronto,3,Park,Coffee Shop,Grocery Store,Athletics & Sports,Sandwich Place,Bistro,Juice Bar,Bank,Japanese Restaurant,Breakfast Spot
13,Downtown Toronto,3,Coffee Shop,Park,Pub,Café,Diner,Sushi Restaurant,Italian Restaurant,Breakfast Spot,Theater,Bakery
22,Central Toronto,3,Café,Sushi Restaurant,Bank,Pharmacy,Coffee Shop,Italian Restaurant,Gastropub,Clothing Store,Skating Rink,Lingerie Store
23,Central Toronto,3,Café,Park,Coffee Shop,Gym / Fitness Center,Trail,Skating Rink,Burger Joint,Liquor Store,Sushi Restaurant,Italian Restaurant
31,West Toronto,3,Café,Park,Coffee Shop,Italian Restaurant,Sushi Restaurant,Bar,Gourmet Shop,Convenience Store,Pharmacy,Bakery
38,East Toronto,3,Park,Coffee Shop,Pizza Place,Brewery,Pet Store,Sushi Restaurant,Italian Restaurant,Burrito Place,Fish & Chips Shop,Bistro


In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Toronto,4,Coffee Shop,Italian Restaurant,Café,Fast Food Restaurant,Pizza Place,Gym,Sushi Restaurant,Dessert Shop,Mexican Restaurant,Bookstore
7,Central Toronto,4,Coffee Shop,Italian Restaurant,Sushi Restaurant,Indian Restaurant,Middle Eastern Restaurant,Café,Pub,Gym,Pizza Place,Dessert Shop
11,Downtown Toronto,4,Gastropub,Café,Diner,Japanese Restaurant,Park,Coffee Shop,Thai Restaurant,Italian Restaurant,Theater,Restaurant
12,Downtown Toronto,4,Coffee Shop,Park,Japanese Restaurant,Men's Store,Café,Gym,Italian Restaurant,Restaurant,Hotel,Gay Bar
14,Downtown Toronto,4,Coffee Shop,Middle Eastern Restaurant,Restaurant,Cosmetics Shop,Clothing Store,Italian Restaurant,Japanese Restaurant,Café,Tea Room,Bakery
15,Downtown Toronto,4,Coffee Shop,Café,Restaurant,Seafood Restaurant,Hotel,Bakery,Breakfast Spot,Cosmetics Shop,BBQ Joint,Clothing Store
16,Downtown Toronto,4,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Beer Bar,Park,Cocktail Bar,Seafood Restaurant,Gastropub
17,Downtown Toronto,4,Coffee Shop,Café,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Ice Cream Shop,Burger Joint,Burrito Place,Bakery,Bookstore
18,Downtown Toronto,4,Café,Coffee Shop,Hotel,Theater,Restaurant,Cosmetics Shop,Steakhouse,Ramen Restaurant,Pizza Place,Gym
19,Downtown Toronto,4,Coffee Shop,Hotel,Café,Italian Restaurant,Restaurant,Aquarium,Park,Brewery,Scenic Lookout,Deli / Bodega
